# Building footprint (plus street network) figure-ground diagrams

Author: [Geoff Boeing](https://geoffboeing.com/)

Use OSMnx to download OpenStreetMap building footprints and visualize them as figure-ground diagrams.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import matplotlib.pyplot as plt
import osmnx as ox
from IPython.display import Image
ox.config(log_console=True)
ox.__version__

'1.0.1'

In [2]:
# configure the inline image display
img_folder = 'images'
extension = 'png'
size = 240

# specify that we're retrieving building footprint geometries
tags = {'building': True}

## Building footprints within the city limits of Piedmont, California

In [64]:
gdf = ox.geometries_from_place('Piedmont, California, USA', tags)
gdf_proj = ox.project_gdf(gdf)
fp = f'./{img_folder}/piedmont_bldgs.{extension}'
fig, ax = ox.plot_footprints(gdf_proj, filepath=fp, dpi=400, save=True, show=False, close=True)
Image(fp, height=size, width=size)

KeyboardInterrupt: 

In [ ]:
# save as a shapefile
gdf_save = gdf.applymap(lambda x: str(x) if isinstance(x, list) else x)
gdf_save.drop(labels='nodes', axis=1).to_file('./data/piedmont_bldgs.gpkg', driver='GPKG')

#### Now let's analyze the size of the building footprints...

In [ ]:
# calculate the area in projected units (meters) of each building footprint, then display first five
areas = gdf_proj.area
areas.head()

In [ ]:
# total area (sq m) covered by building footprints
sum(areas)

In [ ]:
# get the total area within Piedmont's admin boundary in sq meters
place = ox.geocode_to_gdf('Piedmont, California, USA')
place_proj = ox.project_gdf(place)
place_proj.area.iloc[0]

In [ ]:
# what proportion of piedmont is covered by building footprints?
sum(areas) / place_proj.area.iloc[0]

## One and a half square kilometers near the Arc de Triomphe

In [ ]:
point = (40.036123, -83.1441865)
dist = 612
# define a bounding box in San Francisco
north, south, east, west = 37.79, 37.78, -122.41, -122.43

# create network from that bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type='drive_service')
gdf = ox.geometries_from_point(point, tags, dist=dist)
gdf_proj = ox.project_gdf(gdf)
bbox = ox.utils_geo.bbox_from_point(point=point, dist=dist, project_utm=True)
fp = f'./{img_folder}/paris_bldgs.{extension}'
#fig, ax = ox.plot_figure_ground(point=point, dist=dist, network_type=network_type, default_width=default_width,
                   #                 street_widths=street_widths, save=False, show=False, close=True)
fig, ax = ox.plot_footprints(gdf_proj, bbox=bbox, color='w', filepath=fp, dpi=90,
                             save=True, show=False, close=True)
Image(fp, height=size, width=size)

## Street network + building footprints: square-mile visualizations

Plot and save to disk as .png

In [ ]:
# helper funcion to get one-square-mile street networks, building footprints, and plot them
def make_plot(place, point, network_type='drive', dpi=100, dist=2000, default_width=4, street_widths=None):
    fp = f'./{img_folder}/{place}.{extension}'
    gdf = ox.geometries_from_point(point, tags, dist=dist)
    fig, ax = ox.plot_figure_ground(point=point, dist=dist, network_type=network_type, default_width=default_width,
                                    street_widths=street_widths, save=False, show=False, close=True)
    fig, ax = ox.plot_footprints(gdf, ax=ax, filepath=fp, dpi=dpi, save=True, show=False, close=True)

In [ ]:
place = 'portland_buildings'
point = (45.517309, -122.682138)
make_plot(place, point)

In [ ]:
place = 'richmond_district_buildings'
point = (37.781999, -122.472501)
make_plot(place, point)

In [ ]:
place = 'port_au_prince_buildings'
point = (18.522240, -72.347607)
make_plot(place, point, network_type='all', default_width=1, street_widths={'secondary':3})

In [ ]:
place = 'monrovia_liberia_buildings'
point = (6.340236, -10.747255)
make_plot(place, point, network_type='all', default_width=2, street_widths={'primary':3})

In [ ]:
place='hillard_ohio'
point = (40.036123, -83.1441865)
make_plot(place, point)

In [3]:
# helper funcion to get one-square-mile street networks, building footprints, and plot them
def make_plot_2(place,north,south,east,west, network_type='drive', dpi=200, default_width=2, street_widths=None):
    fp = f'./{img_folder}/{place}.{extension}'
    tags = {'building': True}
        
    gdf_building = ox.geometries_from_bbox(north, south, east, west, tags={'building':True})
    gdf_nature = ox.geometries_from_bbox(north, south, east, west, tags={'natural':True,'landuse':'forest','landuse':'grass'})
    gdf_water = ox.geometries_from_bbox(north, south, east, west, tags={'natural':'water'})
    fig, ax= ox.plot_footprints(gdf_nature, color='green', filepath=fp, dpi=dpi, save=True, show=False, close=True)    
    fig, ax= ox.plot_footprints(gdf_water, color='blue', ax=ax, filepath=fp, dpi=dpi, save=True, show=False, close=True)    
    fig, ax = ox.plot_footprints(gdf_building,ax=ax, filepath=fp, dpi=dpi, save=True, show=False, close=True)
    print('fin')


In [4]:
place='hillard_ohio_latest'
make_plot_2(place, north=40.067995,south=40.004251,west=-83.195513,east=-83.092860 ,street_widths={'primary':2})

fin


In [75]:
place='hillard_ohio_latest_4'
make_plot_2(place, north=40.0103392,south=40.0096992,west=-83.0143545,east=-83.0124745,street_widths={'primary':2})

/Users/dkarakay/opt/miniconda3/envs/gm-with-osm/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NetworkXPointlessConcept: Connectivity is undefined for the null graph.

In [ ]:
def getting_boundary_coordinates(lat: float, long: float) -> (float, float, float, float):
    lat_coef = 0.00032
    long_coef = 0.00094

    north = lat + lat_coef
    south = lat - lat_coef
    east = long + long_coef
    west = long - long_coef

    return north, south, west, east

In [54]:
zoom = 20
getting_boundary_coordinates(lat=40.000133,long=-83.0130406)


(40.000453, 39.999812999999996, -83.0139806, -83.0121006)

In [28]:
print(north)

40.0004801


In [29]:
print(south)

39.9997859


In [30]:
print(west)

-83.01350228939043


In [31]:
print(east)

-83.01257891060956


In [55]:
for i in range(2):
    print(i)


0
1
